<a href="https://colab.research.google.com/github/CodingTomo/TE-UrbanSound8K/blob/master/MelSpectrum_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torchaudio

     |████████████████████████████████| 6.7MB 5.4MB/s 


In [2]:
import os
import pandas as pd
import torchaudio
import torch
import librosa

In [3]:
metadata = pd.read_csv('/content/drive/Shared drives/TE-UrbanSound/UrbanSound8K/metadata/UrbanSound8K.csv')
SAMPLE_RATE = 8000
MELKWARGS = {'n_fft': 512, 'hop_length': 128}
N_MELS = 90
N_WAVEFORM = metadata.shape[0]

In [4]:
LIBROSA = 'OFF'
TORCHAUDIO = 'ON'

In [5]:
if LIBROSA == 'ON':
  data = {'name': [],
          'log_mel_spectrogram': [],
          'label': []
          }
  i=0

  for root, dirs, files in os.walk('/content/drive/Shared drives/TE-UrbanSound/UrbanSound8K'):
      for name in files:
          if name.endswith(".wav"):
              complete_path = root + "/" + name
              waveform, sr = librosa.load(complete_path, sr=SAMPLE_RATE, mono=True)
              mel_spectrogram = librosa.feature.melspectrogram(waveform, 
                                                                sr=sr, 
                                                                n_fft=MELKWARGS['n_fft'], 
                                                                hop_length=MELKWARGS['hop_length'],
                                                                n_mels=N_MELS)
              log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
              data['name'].append(name)
              data['log_mel_spectrogram'].append(log_mel_spectrogram)
              data['label'].append(metadata.loc[metadata['slice_file_name'] == name]['classID'].iloc[0])
              i=i+1
              if i%100==0:
                print('-'*40)
                print('Processed {:.0%}'.format(i/N_WAVEFORM))

In [6]:
def stereo_to_mono(waveform):
    waveform_mono = torch.mean(waveform, dim=0, keepdim=True)
    return waveform_mono

In [7]:
def resampling_8k(waveform, sr, SAMPLE_RATE):
    waveform = torchaudio.transforms.Resample(new_freq=SAMPLE_RATE, orig_freq=sr)(waveform)
    return waveform


In [11]:
if TORCHAUDIO == 'ON':
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  with torch.cuda.device(0):
    data = {'name': [],
            'log_mel_spectrogram': [],
            'label': []
            }
    i=0

    for root, dirs, files in os.walk('/content/drive/Shared drives/TE-UrbanSound/UrbanSound8K'):
        for name in files:
            if name.endswith(".wav"):
                complete_path = root + "/" + name
                waveform, sr = torchaudio.load(complete_path)
                waveform.to(device)
                if waveform.shape[0] != 1:
                    waveform = stereo_to_mono(waveform)
                if sr != SAMPLE_RATE:
                    waveform = resampling_8k(waveform, sr, SAMPLE_RATE)
                mel_spectrogram = torchaudio.transforms.MelSpectrogram(
                                                                    sample_rate=SAMPLE_RATE,
                                                                    n_fft=MELKWARGS['n_fft'], 
                                                                    hop_length=MELKWARGS['hop_length'],
                                                                    n_mels=N_MELS)(waveform)
                log_mel_spectrogram = torchaudio.transforms.AmplitudeToDB(mel_spectrogram)
                data['name'].append(name)
                data['log_mel_spectrogram'].append(log_mel_spectrogram)
                data['label'].append(metadata.loc[metadata['slice_file_name'] == name]['classID'].iloc[0])
                i=i+1
                if i%100==0:
                  print('-'*40)
                  print('Processed {:.0%}'.format(i/N_WAVEFORM))

KeyboardInterrupt: ignored